In [65]:
from openai import OpenAI
import json
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()


name = "gpt-3.5-turbo memory-1 att-user-assistant"
model = "gpt-3.5-turbo"               # important options: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
n = 1
memory = 1
stable_system_message = False
assistant_transform = True    
assistant_transform_type = "user"     # options: "user", "user-assistant", "system"

first_prompt = [
  #{"role": "system", "content": ""},
  {"role": "user", "content": ""}
]

def create_completions(name,n,memory,stable_system_message,assistant_transform,assistant_transform_type,first_prompt):

  # Load completions from json, if continuing with a file generated in the past

  try:
    with open(f"{name}.json", "r") as json_file:
      completions=json.load(json_file)

  # If not, create a new completions database

  except FileNotFoundError:
    completions = []

  # Collect messages generated so far

  messages = first_prompt
  for completion in completions:
      message = completion["choices"][0]["message"]
      messages.append({"role": message["role"], "content": message["content"]}   
      )

  # Pick prompt messages

  if memory >= len(messages):
      prompt_messages = messages
  else:
      #print(f"Memory smaller than the entire conversation. Forgetting {len(messages)-memory} messages")
      prompt_messages = []
      if stable_system_message:
        prompt_messages.append(first_prompt[0])
        print(prompt_messages)
        print("stable_system_message")
      prompt_messages = prompt_messages+messages[(len(messages)-memory):]

  # Transform assistant role to assistant and user, user, or system roles

  if assistant_transform:
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages):
        if prompt_messages[-i]["role"] == "assistant":
          prompt_messages[-i]["role"] = "user"
        i += 2
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type


  # Send to OpenAI API to get a new completion

  completion = client.chat.completions.create(
        model=model,
        messages = prompt_messages
      )
  print(f"Prompt messages sent to OpenAI API:{prompt_messages}")

  # Update database and save it to the right json file

  completions.append(completion.model_dump())
  name = f"{model} memory-{memory} {assistant_transform_type}"
  with open(f"{name}.json", "w") as json_file:
      json.dump(completions,json_file)

create_completions(name,n,memory,stable_system_message,assistant_transform,assistant_transform_type,first_prompt)


## Display data

# Load the JSON file

file_path = f"{name}.json"
with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the JSON data in completions

normalized_data = pd.json_normalize(data, record_path='choices', meta=[
    'id', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 
    ['usage', 'completion_tokens'], ['usage', 'prompt_tokens'], ['usage', 'total_tokens']], sep='.',
    meta_prefix='', record_prefix='choices.')

# Make more important columns more visible

normalized_data = normalized_data[['choices.message.content', 'model', 
       'usage.completion_tokens', 'usage.prompt_tokens', 'usage.total_tokens', 'id', 
       'choices.finish_reason', 'choices.index', 'choices.logprobs', 
       'choices.message.role', 'choices.message.function_call', 'choices.message.tool_calls', 
       'created', 'object', 'service_tier', 'system_fingerprint'
       ]]

# Display

text = normalized_data["choices.message.content"].iloc[-1]
display_markdown(text, raw=True)
 

Memory smaller than the entire conversation. Forgetting 3 messages
4
1
Prompt messages sent to OpenAI API:[{'role': 'user', 'content': 'Hello! I have a question about financial planning. Can you provide some guidance on how to create a budget and save for the future?'}]


Of course! Creating a budget and saving for the future are two crucial steps in financial planning. Here are some tips to help you get started:

1. Track your expenses: Start by tracking all of your expenses for one month. This will give you a clear picture of where your money is going and help you identify areas where you can cut back.

2. Create a budget: Based on your expenses, create a budget that outlines how much money you will allocate towards various categories such as rent, groceries, utilities, entertainment, and savings.

3. Set financial goals: Determine what you are saving for - whether it's an emergency fund, a vacation, a down payment on a house, or retirement. Setting specific financial goals will help you stay motivated and focused on saving.

4. Automate your savings: Setting up automatic transfers from your checking account to a savings account can help you save consistently without having to think about it.

5. Cut back on expenses: Look for ways to reduce your spending, such as cutting back on dining out, canceling unnecessary subscriptions, or finding cheaper alternatives for everyday expenses.

6. Prioritize debt repayment: If you have high-interest debt, such as credit card debt, prioritize paying it off to avoid accruing more interest.

7. Review and adjust your budget regularly: Periodically review your budget to track your progress towards your financial goals and make necessary adjustments to ensure you stay on track.

By following these steps and staying disciplined with your budget, you'll be well on your way to saving for the future and achieving financial stability. Let me know if you have any other questions or need further guidance.

In [66]:
normalized_data["choices.message.content"].iloc[-2]

'Hello! I have a question about financial planning. Can you provide some guidance on how to create a budget and save for the future?'

In [67]:
normalized_data["choices.message.content"].iloc[-3]

"Hello! I'm here to assist you with any questions or tasks you may have. How can I help you today?"